In [47]:
import os
from dotenv import load_dotenv

load_dotenv()  # Reads .env in current directory

CLIENT_ID       = os.getenv("TRAVELPORT_APPLICATION_KEY")
CLIENT_SECRET   = os.getenv("TRAVELPORT_APPLICATION_SECRET")
USERNAME        = os.getenv("TRAVELPORT_USERNAME")
PASSWORD        = os.getenv("TRAVELPORT_PASSWORD")
ACCESS_GROUP    = os.getenv("TRAVELPORT_ACCESS_GROUP")

OAUTH_URL       = "https://oauth.pp.travelport.com/oauth/oauth20/token"
CATALOG_URL     = "https://api.pp.travelport.com/11/air/catalog/search/catalogproductofferings"


In [48]:
import requests

def fetch_password_token():
    data = {
        "grant_type":    "password",
        "username":      USERNAME,
        "password":      PASSWORD,
        "client_id":     CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope":         "openid"
    }
    resp = requests.post(
        OAUTH_URL,
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data=data
    )
    resp.raise_for_status()
    return resp.json()["access_token"]

token = fetch_password_token()
print(f"🔑 Token obtained: {token[:20]}…")

🔑 Token obtained: eyJhbGciOiJkaXIiLCJl…


In [49]:
headers = {
    "Accept":                       "application/json",
    "Content-Type":                 "application/json",
    "Accept-Encoding":              "gzip, deflate",
    "Cache-Control":                "no-cache",
    "Authorization":                f"Bearer {token}",
    "XAUTH_TRAVELPORT_ACCESSGROUP": ACCESS_GROUP,
    "Accept-Version":               "11",
    "Content-Version":              "11",
}
print("📋 Headers built")

📋 Headers built


In [85]:
from datetime import datetime, timedelta
import json

# Compute or set your travel dates here:
departure_date = (datetime.utcnow() + timedelta(days=30)).strftime("%Y-%m-%d")
return_date    = (datetime.utcnow() + timedelta(days=37)).strftime("%Y-%m-%d")
from_city = "LHE"
to_city = "ATH"
carrier_list = ["QR", "EY", "GF", "SV"]

# Now build the payload using those real date strings:
payload = {
    "@type": "CatalogProductOfferingsQueryRequest",
    "CatalogProductOfferingsRequest": {
        "@type": "CatalogProductOfferingsRequestAir",
        "maxNumberOfUpsellsToReturn": 1,
        "contentSourceList": ["GDS"],
        "PassengerCriteria": [
            {
                "@type": "PassengerCriteria",
                "number": 1,
                "age": 25,
                "passengerTypeCode": "ADT"
            }
        ],
        "SearchCriteriaFlight": [
            {
                "@type": "SearchCriteriaFlight",
                "departureDate": departure_date,   # injected here
                "From": {"value": from_city},
                "To":   {"value": to_city}
            },
            {
                "@type": "SearchCriteriaFlight",
                "departureDate": return_date,      # and here
                "From": {"value": to_city},
                "To":   {"value": from_city}
            }
        ],
        "SearchModifiersAir": {
            "@type": "SearchModifiersAir",
            "CarrierPreference": [
                {
                    "@type": "CarrierPreference",
                    "preferenceType": "Preferred",
                    "carriers": carrier_list
                }
            ]
        },
        "CustomResponseModifiersAir": {
            "@type": "CustomResponseModifiersAir",
            "SearchRepresentation": "Journey"
        }
    }
}

print(json.dumps(payload, indent=2))


{
  "@type": "CatalogProductOfferingsQueryRequest",
  "CatalogProductOfferingsRequest": {
    "@type": "CatalogProductOfferingsRequestAir",
    "maxNumberOfUpsellsToReturn": 1,
    "contentSourceList": [
      "GDS"
    ],
    "PassengerCriteria": [
      {
        "@type": "PassengerCriteria",
        "number": 1,
        "age": 25,
        "passengerTypeCode": "ADT"
      }
    ],
    "SearchCriteriaFlight": [
      {
        "@type": "SearchCriteriaFlight",
        "departureDate": "2025-08-16",
        "From": {
          "value": "LHE"
        },
        "To": {
          "value": "ATH"
        }
      },
      {
        "@type": "SearchCriteriaFlight",
        "departureDate": "2025-08-23",
        "From": {
          "value": "ATH"
        },
        "To": {
          "value": "LHE"
        }
      }
    ],
    "SearchModifiersAir": {
      "@type": "SearchModifiersAir",
      "CarrierPreference": [
        {
          "@type": "CarrierPreference",
          "preferenceType": "P

In [83]:
response = requests.post(CATALOG_URL, headers=headers, json=payload)
try:
    response.raise_for_status()
    result = response.json()
    import json
    print(json.dumps(result, indent=2))
except requests.HTTPError as e:
    print("❌ Request failed:", e)
    print(response.status_code, response.text)

{
  "CatalogProductOfferingsResponse": {
    "@type": "CatalogProductOfferingsResponse",
    "transactionId": "ebf48f6d-f633-4cb6-a9fe-ca7111f7a983",
    "CatalogProductOfferings": {
      "@type": "CatalogProductOfferings",
      "CatalogProductOffering": [
        {
          "@type": "CatalogProductOffering",
          "sequence": 1,
          "id": "o1",
          "Departure": "LHE",
          "Arrival": "ATH",
          "Brand": [
            {
              "@type": "BrandID",
              "BrandRef": "b0"
            },
            {
              "@type": "BrandID",
              "BrandRef": "b1"
            }
          ],
          "ProductBrandOptions": [
            {
              "@type": "ProductBrandOptions",
              "flightRefs": [
                "s9",
                "s10"
              ],
              "ProductBrandOffering": [
                {
                  "@type": "ProductBrandOffering",
                  "Brand": {
                    "@type": "BrandI